In [ ]:
"""
This program simulates the random rolls for Coltzan's shrine. It assumes perfect conditions to get the maximum score:
- Time condition: go on the 15th of the month at midnight
- Active pet condition: have a pet with 201 or more intelligence and 10 or less HP

The result of the simulation is stored in a file 'scores.npy'
You can run simulations for a custom number of seconds
"""
import numpy as np
from random import random, choice
from datetime import datetime, timedelta

VERSION = 2
FILENAME = f"scores_{VERSION}.npy"

def choice_inclusive(low, high):
    return choice(range(low, high + 1))
    
def odds(v):
    return random() < v

def get_scores():
    try:
        return np.load(FILENAME)
    except FileNotFoundError:
        scores = np.zeros(100, dtype=int)
        np.save(FILENAME, scores)
        return scores

def run(max_seconds):
    """Function that computes a score for Coltzan. It takes into account all the random bonuses but not the two fixed bonuses. The function runs for max_sceonds seconds"""
    scores = get_scores()
    start = datetime.now()
    while (datetime.now() - start).seconds < max_seconds:
            # Start with a random prize value between 1 and 7
            score = choice_inclusive(1, 7)

            # On the 15th you roll three times with 20% odds of getting a score between 1 and 12
            for _ in range(3):
                if odds(1/5):
                    score = score + choice_inclusive(1, 12)

            # pet hp bonus
            score = score + choice_inclusive(1, 5)
            scores[score] += 1
    np.save(FILENAME, scores)

In [ ]:
# Run simulations for the specified number of seconds
run(600)

In [ ]:
###############
## Analaysis ##
###############
scores = get_scores()

# You can always get 12 bonus points for having a pet
# with 201 or more intelligence (+6) and going at midnight (+6)
BONUS = 12

real_scores = np.concatenate([
    np.zeros(BONUS, dtype=int),
    scores
])


# Make sure prize value is between 1 and 59
real_scores[59] = real_scores[59:].sum()
real_scores[60:] = 0

# Realistically your lowest score is 14 and your highest score
# is 59. You get 6 points for going at midnight, 6 bonus points
# for intelligence, at least 1 point for the initial roll and
# at least 1 point for the pet HP roll.
assert real_scores[0:14].sum() == 0

# Ensure the program ran long enough to get each score at least 5 times
assert real_scores[14:59].min() > 5

# Ensure you do not have scores above 59
assert real_scores[60:].sum() == 0

from matplotlib import pyplot as plt
# For the presentation, divide by the lowest occuring number
# so you have a 1 out of xxxx times presentation
real_scores = real_scores / real_scores[14:60].min()
print("| score | odds |")
print("| --- | --- |")

for score, count in enumerate(real_scores):
    if count == 0:
        continue
    print(f"| {score} | {count:0.0f} out of {real_scores.sum():0.0f} times |")
print()
print(f"({scores.sum():,d} simulations)")